# Исследование надёжности заёмщиков #  
### Описание проекта ###
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Этап 1. Открытие таблицы и изучение общей информации о данных ###

#### Импорт библиотек ####

In [1]:
import pandas as pd

from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

from pymystem3 import Mystem
m = Mystem()


Прочитаем файл *data.csv* и сохраним его в переменной *data*. 

In [2]:
data = pd.read_csv('data.csv')

Получение первых 10 строк таблицы

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы *df*.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Имеется 12 столбцов и 21525 строк. Здесь присутствует 7 столбцов с числовыми данными и 5 столбцов с объектами, при этом в столбцах *days_employed* и *total_income* есть NaN объекты.

Подробно разберём, какие в *data* столбцы и какую информацию они содержат:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

#### Вывод ####

Необходимо выяснить причину отсуствия значений. Проверить достоверность данных в каждом столбце

### Этап 2. Предобработка данных ###

#### Определим и заполним пропущенные значения####

Выведем все уникальные значения столбца *income_type*

In [5]:
print('Уникальные значения в столбце "income_type":',sep = '\n')
print(*data['income_type'].unique(), sep=", ")

Уникальные значения в столбце "income_type":
сотрудник, пенсионер, компаньон, госслужащий, безработный, предприниматель, студент, в декрете


Из представленных значений особое внимание заслуживают студенты. Быть может NaN значения относятся к студентам (как правило люди без опыта работы) и тогда мы легко сделаем замену NaN на 0.
  
  
Выведем сгруппированную таблицу по типу занятости  только с NaN объектами в столбце *days_employed*

In [6]:
(data[data['days_employed'].isnull()]).groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,0,147,147,147,147,147,147,147,0,147
компаньон,508,0,508,508,508,508,508,508,508,0,508
пенсионер,413,0,413,413,413,413,413,413,413,0,413
предприниматель,1,0,1,1,1,1,1,1,1,0,1
сотрудник,1105,0,1105,1105,1105,1105,1105,1105,1105,0,1105


Студентов среди них нет. Предположим, что кто-то только устраивается на работу, где-то человеческая ошибка в занесение данных, кто-то работает неофициально и не может принести, подтверждающий документ. Мы можем посчитать среднее *days_employed* для пенсионеров (учитывая пол и период с 18 и до 60/65лет).  
Все же заполним эти пробелы 0 так мы отрабатываем применение инструментов.

 Но перед этим выполним небольшую проверку, проверим соотвествует ли все NaN значения в столбце *days_employed* всем NaN значениям столбца *total_income*

In [7]:
count_nan_data=data.isnull().sum()

## цикл для сверки двух строк
count_nan = 0
for i in range(data.shape[0]): 
    if (pd.isna(data.loc[i,'days_employed']))& pd.isna(data.loc[i,'total_income']):
        count_nan += 1

if count_nan_data['days_employed'] == count_nan:
    print('Все NaN значения в столбце "days_employed" соответствуют всем NaN значениям столбца "total_income" ')
else:
    print('Не все NaN значения в столбце "days_employed" соответствуют всем NaN значениям столбца "total_income" ')

Все NaN значения в столбце "days_employed" соответствуют всем NaN значениям столбца "total_income" 


Проверка подтвердила соотвествие.Заменим пропущенные значения NaN значения нулями

In [8]:
data = data.fillna(0)

После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Заменим вещественный тип данных на целочисленный####

В столбцах *days_employed* и *total_income* выполним замену типа данных с float64 на int64 методом *astype()*

In [10]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')


Убедимся в замене типа данных

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Выполним проверку достоверности значений в каждом столбце. Удалим дубликаты ####

Рассмотрим  столбец *days_employed* 

In [12]:
data['days_employed'].head(15)

0      -8437
1      -4024
2      -5623
3      -4124
4     340266
5       -926
6      -2879
7       -152
8      -6929
9      -2188
10     -4171
11      -792
12         0
13     -1846
14     -1844
Name: days_employed, dtype: int64

 В столбце как "+" так и "-" значения. Так же надо оценить адекватность значений. Рассмотрим   "+" и "-" значения отдельно друг от друга
 

In [13]:
print('Максимальное положительное значение  в столбце "days_employed":',data.loc[data.loc[:,'days_employed'] >0]['days_employed'].max())
print('Минимальное положительное значение  в столбце "days_employed":',data.loc[data.loc[:,'days_employed'] >0]['days_employed'].min())

Максимальное положительное значение  в столбце "days_employed": 401755
Минимальное положительное значение  в столбце "days_employed": 328728


Положительные данные некорректные, человек не может столько работать. 

In [14]:
print('Максимальное отрицательное значение  в столбце "days_employed":',data.loc[data.loc[:,'days_employed']<0]['days_employed'].max())
print('Минимальное отрицательное значение  в столбце "days_employed":',data.loc[data.loc[:,'days_employed'] <0]['days_employed'].min())

Максимальное отрицательное значение  в столбце "days_employed": -24
Минимальное отрицательное значение  в столбце "days_employed": -18388


Отрицательные данные  выглядят приличнее *(abs(-18388.9 /365 )= примерно 51 год)*

С положительными данными работать нет возможности, отрицательные имеют хоть какую-то логичность. Позже проверю кому  принадлежит максимальное значение в столбце *days_employed*.

Оценим объем данных, которые занимают данные с положительными значениями в столбце *days_employed*.

In [15]:
print('Кол-во отрицательных значений в столбце "days_employed":', data.loc[data.loc[:,'days_employed'] <0]['days_employed'].count())
print('Кол-во положительных значений в столбце "days_employed":', data.loc[data.loc[:,'days_employed'] >0]['days_employed'].count())
print('Кол-во нулевых значений в столбце "days_employed":', data.loc[data.loc[:,'days_employed'] == 0]['days_employed'].count())

Кол-во отрицательных значений в столбце "days_employed": 15906
Кол-во положительных значений в столбце "days_employed": 3445
Кол-во нулевых значений в столбце "days_employed": 2174


Положительные значения занимают около 16%-это большой процент, но необходимо удалить строки с необъективными значениями в данном стобце.

In [16]:
data = data[data['days_employed'] <= 0]
data['days_employed']=abs(data['days_employed'])

Рассмотрим столбец *children*

In [17]:
print('Минимальное значение детей в столбце "children":', data['children'].min())
print('Максимальное значение детей в столбце "children":', data['children'].max())

Минимальное значение детей в столбце "children": -1
Максимальное значение детей в столбце "children": 20


Определим кол-во детей  со значением "-1" и "20"

In [18]:
print('Количество детей в столбце "children" со значением "-1":',(data.loc[data.loc[:,'children'] == -1]['children']).count())
print('Количество детей в столбце "children" со значением "20":',(data.loc[data.loc[:,'children'] == 20]['children']).count())

Количество детей в столбце "children" со значением "-1": 40
Количество детей в столбце "children" со значением "20": 69


 В столбце 'children' встретились некорректные значения "-1" и аномальные "20".
 * "-1" можно было превратить в 0
 * "20" следует удалить - это аномальный выброс.

Избавимся от таких значений:

In [19]:
data = data[data['children'] >= 0]
data = data[data['children'] < 20]

Проверю остальные значения 

In [20]:
print('Оставшиеся уникальные значения в столбце "children":', data['children'].unique())

Оставшиеся уникальные значения в столбце "children": [1 0 3 2 4 5]


Чтобы учесть дубликаты, все символы в строках приводят к нижнему регистру вызовом метода lower():


In [22]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

Рассмотрим столбец *education*

In [23]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Столбец  *education* без замечаний

Рассмотрим столбец *family_status*

In [24]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'в разводе',
       'не женат / не замужем', 'вдовец / вдова'], dtype=object)

Столбец  *family_status* без замечаний

Рассмотрим столбец *gender*

In [25]:
data['gender'].unique()

array(['f', 'm', 'xna'], dtype=object)

Столбец 'gender' имеет неизвестное значение *'xna'*. Выясним номер строки 

In [26]:
print(data.loc[data.loc[:,'gender'] == 'xna']['gender'])

10701    xna
Name: gender, dtype: object


Строку с полом "xna" можно как удалить, так и придать любое значение из F or M

In [27]:
data.loc[data['gender'] == 'xna', 'gender'] = 'f' 
print(data.loc[10701])

children                               0
days_employed                       2358
dob_years                             24
education            неоконченное высшее
education_id                           2
family_status           гражданский брак
family_status_id                       1
gender                                 f
income_type                    компаньон
debt                                   0
total_income                      203905
purpose             покупка недвижимости
Name: 10701, dtype: object


Строка *10701* имеет адекватные все остальные параметры.

Рассмотрим столбец *income_type*

In [28]:
data['income_type'].unique()

array(['сотрудник', 'компаньон', 'пенсионер', 'госслужащий',
       'предприниматель', 'студент', 'в декрете'], dtype=object)

Столбец *income_type* без замечаний

Рассмотрим столбец *purpose*

In [29]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'операции с жильем', 'образование',
       'на проведение свадьбы', 'покупка жилья для семьи',
       'покупка недвижимости', 'покупка коммерческой недвижимости',
       'сыграть свадьбу', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку своего автомобиля',
       'на покупку подержанного автомобиля',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'операции с коммерческой недвижимостью', 'свой автомобиль',
       'сделка с автомобилем', 'профильное образование',
       'высшее об

Столбец  *purpose* имеет много схожих целей.

Избавимся от дубликатов

In [30]:
print(data.duplicated().sum()) 
data.drop_duplicates(inplace = True)  

71


Приведем индексы в порядок после удаления

In [31]:
data.reset_index(drop=True, inplace=True)

#### Выполним выделение лемм в значениях столбца *purpose* ####

Создадим функцию для создания лемм

In [32]:
def funk_lemma(row):
    lemma = m.lemmatize(row)
    return lemma

Добавим столбец *purpose_lemma*,  куда определим цели из столбца *purpose*, после применения функции *funk_lemma*

In [33]:
data['purpose_lemma'] = data['purpose'].apply(funk_lemma)

Преобразуем все значения из столбца *purpose_lemma* в удобный вариант для расчета. Преобразуем  в единный список с разбивкой по словам.

In [34]:
# добавим в список все леммы
lemma_list = data['purpose_lemma']
lemma_list_out = []
for i in lemma_list:
    for j in i:
        lemma_list_out.append(j)
lemma_list_out

['покупка',
 ' ',
 'жилье',
 '\n',
 'приобретение',
 ' ',
 'автомобиль',
 '\n',
 'покупка',
 ' ',
 'жилье',
 '\n',
 'дополнительный',
 ' ',
 'образование',
 '\n',
 'покупка',
 ' ',
 'жилье',
 '\n',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 '\n',
 'образование',
 '\n',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 '\n',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 '\n',
 'покупка',
 ' ',
 'недвижимость',
 '\n',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 '\n',
 'сыграть',
 ' ',
 'свадьба',
 '\n',
 'приобретение',
 ' ',
 'автомобиль',
 '\n',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 '\n',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 '\n',
 'недвижимость',
 '\n',
 'строительство',
 ' ',
 'недвижимость',
 '\n',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 '\n',
 'недвижимость',
 '\n',
 'приобретение',
 ' ',
 'автомобиль',
 '\n',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 '\n',
 'сыграть',
 '

Посчитаем самые популярные слова, чтобы обобщить цели кредитов.

In [35]:
from collections import Counter
Counter(lemma_list_out)

Counter({'покупка': 4934,
         ' ': 28021,
         'жилье': 3748,
         '\n': 17900,
         'приобретение': 382,
         'автомобиль': 3567,
         'дополнительный': 738,
         'образование': 3336,
         'операция': 2178,
         'с': 2436,
         'на': 1842,
         'проведение': 637,
         'свадьба': 1939,
         'для': 1077,
         'семья': 530,
         'недвижимость': 5310,
         'коммерческий': 1092,
         'сыграть': 633,
         'жилой': 1026,
         'строительство': 1560,
         'собственный': 528,
         'свой': 1897,
         'подержать': 387,
         'со': 526,
         'заниматься': 769,
         'сделка': 784,
         'подержанный': 403,
         'получение': 1087,
         'высокий': 1153,
         'профильный': 362,
         'сдача': 547,
         'ремонт': 513})

Добавлю в словарь самые популярные обобщенные цели

In [36]:
dict_purpose = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']
dict_purpose

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']

Придадим категории из словаря *dict_purpose* для каждой цели из столбца *purpose*

In [37]:
def category(data):
    if dict_purpose[0] in data:
        return dict_purpose[0]
    elif dict_purpose[1] in data:
        return dict_purpose[1]
    elif dict_purpose[2] in data:
        return dict_purpose[2]
    elif dict_purpose[3] in data:
        return dict_purpose[3]
    else:
        return dict_purpose[4]
    
data['category'] = data['purpose_lemma'].apply(category)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",жилье


#### Вывод ####

Предобработка данных выполнена.Теперь таблица полностью читаема и в ней отсуствуют некорректные значения. Можно переходить к исследования и выявлению зависимостей

#### Этап 3. Отвечаем  на вопросы ####

1.Есть ли зависимость между наличием детей и возвратом кредита в срок?


Определим процент людей с детьми, которые имеют задолженности

In [38]:
data_with_children = data[data['children'] != 0]
with_children_debt = data_with_children['debt'].sum() / len(data_with_children)
print('Процент людей с детьми, которые имеют задолжности: {:.1%}'.format(with_children_debt))

Процент людей с детьми, которые имеют задолжности: 9.4%


Определим среднюю зарплату среди них

In [39]:
print("Средняя зарплата людей с детьми:",data_with_children['total_income'].mean())

Средняя зарплата людей с детьми: 155373.24472210254


Определим процент людей без детей, которые имеют задолженности

In [40]:
data_without_children = data[data['children'] == 0]
without_children_debt = data_without_children['debt'].sum() / len(data_without_children)
print('Процент людей без детей, которые имеют задолжности: {:.1%}'.format(without_children_debt))

Процент людей без детей, которые имеют задолжности: 8.2%


Определим среднюю зарплату среди них

In [41]:
print("Средняя зарплата людей без детей:",data_without_children['total_income'].mean())

Средняя зарплата людей без детей: 152612.84191277315


### Вывод: ###
Процент людей с детьми, которые имеют задолжности 9.4%, а без детей 8.2%. На первый взягляд разница не такая большая, чтобы делать выводы. Однако при сравнение средней зарплаты в каждой из этих групп видно, что она примерно одинаковая- 155 тысяч против 152 тысячи.  
Зависимость все же есть. Человек без детей более благонадежен для банка.

2.Есть ли зависимость между семейным положением и возвратом кредита в срок?

Подготовим сводную таблицу для анализа:

In [42]:
data_pivot_FamilyStatus_and_debt = data.pivot_table(index='family_status', columns=['debt'], values='family_status_id', aggfunc='count')
data_pivot_FamilyStatus_and_debt['%'] = (data_pivot_FamilyStatus_and_debt[1] / data_pivot_FamilyStatus_and_debt[0]) * 100
print(data_pivot_FamilyStatus_and_debt)

debt                      0    1          %
family_status                              
в разводе               919   73   7.943417
вдовец / вдова          440   31   7.045455
гражданский брак       3202  356  11.118051
женат / замужем        9565  829   8.667015
не женат / не замужем  2224  261  11.735612


Проверим средний возраст в некоторых группах:

In [43]:
data_age= data[data['family_status'] =='гражданский брак']['dob_years'].mean()
print('Средний возраст среди людей, живущих в гражданском браке',data_age)

data_age= data[data['family_status'] =='женат / замужем']['dob_years'].mean()
print('Средний возраст среди людей, живущих в браке',data_age)

data_age= data[data['family_status'] =='вдовец / вдова']['dob_years'].mean()
print('Средний возраст вдовцов',data_age)

Средний возраст среди людей, живущих в гражданском браке 39.359190556492415
Средний возраст среди людей, живущих в браке 40.79767173369252
Средний возраст вдовцов 52.11677282377919


#### Вывод ####
* Кто не был в официальном браке более склонен к задолжностям 11.7%
* Люди в разводе 8% и вдовцы 7% менее склонны
* Женатые 8.6% и люди,живущие в гражданском браке 11.1%

Зависимость однозначно есть. Наиболее предпочтительно для банка давать кредит женатым 8.6% и людям в разводе 8%. К вдовцам с 7% стоит присмотреться внимательно, средний возраст в группе 52 года. Менее благонадежные для банка люди, живущие в гражанском браке 11.1% и холостяки 11.7%

3.Как разные цели кредита влияют на его возврат в срок?

Подготовим сводную таблицу для анализа:

In [44]:
data_pivot_purpose_and_debt = data.pivot_table(index='category', columns='debt', values='purpose', aggfunc='count')
data_pivot_purpose_and_debt['%'] = (data_pivot_purpose_and_debt[1] / data_pivot_purpose_and_debt[0]) * 100
print(data_pivot_purpose_and_debt)

debt             0    1          %
category                          
автомобиль    3210  357  11.121495
жилье         3473  275   7.918226
недвижимость  4883  427   8.744624
образование   3009  327  10.867398
свадьба       1775  164   9.239437


#### Вывод ####
* Люди, берущие кредит на дорогостоящие вещи менее склонны к задолженностям: 7.9% (жилье) и 8.7% (недвижимость)
* На свадьбу 9.2%  и образование 10.8%
* На автомобиль 11.12%


4.Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выведем максимальную и минимальную з/п

In [45]:
print(data['total_income'].max())
print(data['total_income'].min())

2265604
0


Напишем функцию, которая будет определять диапазон з/п. 

In [46]:
def income(data):
    if data == 0:
        return '0'
    elif data <= 20000:
        #очень низкий доход
        return '0-20000'
    elif data <= 50000:
        # средний доход
        return '20000-50000'
    elif data <= 80000:
         # выше среднего доход
        return '50000-80000'
    elif data <= 120000:
        # выше среднего доход
        return '80000-120000'
    elif data <= 200000:
        # высокий доход
        return '120000-200000'
    else:
       # очень высокий доход
        return '> 200000'
    
data['total_income_pazprez'] = data['total_income'].apply(income)
print(data.loc[:, ['total_income', 'total_income_pazprez']] .head(5))

   total_income total_income_pazprez
0        253875             > 200000
1        112080         80000-120000
2        145885        120000-200000
3        267628             > 200000
4        255763             > 200000


Подготовим сводную таблицу для анализа:

In [47]:
data_pivot_income = data.pivot_table(index='total_income_pazprez', columns='debt', values='total_income', aggfunc='count')
#считаем процент должников
data_pivot_income['ratio %'] = (data_pivot_income[1] / data_pivot_income[0]) * 100
print(data_pivot_income)

debt                     0    1    ratio %
total_income_pazprez                      
0                     1921  170   8.849558
120000-200000         5660  590  10.424028
20000-50000            178   14   7.865169
50000-80000           1161  120  10.335917
80000-120000          3270  331  10.122324
> 200000              4160  325   7.812500


#### Вывод ####
* Самые больше задолженности у людей с очень низким доходом (12-20к)-10.4%
* Самые маленькие задолженности у людей с очень высоким доходом (>200к)-7.86%
* Кто имеет заработок выше среднего(50-80к) и (80-120к)более склонны к задолжностям: 10.3% и 10.1%
* А вот люди со средним доходом (20-50к)-молодцы:7.81%  

    Для банка наиболее удобнее работать с людьми со средним(20-50к) и  очень восоким доходом (>200к)

###  Общий вывод ###
 Сформулируем для банка  образ самого благонадежного клиента:
* Женатый(замужем) или разведенный человек 
* Без детей
* Со средним(20-50к) или очень высоким(>200к) доходом
* С целью получения кредита на что-то очень дорогостоящее( недвижимость,жилье)

Если в таблицу добавить столбцы срок кредита и сумма кредита,то исследование могли быть более глобальные
